<h1>Process Data</h1>
<h4>Haley Johnson</h4>

<p>Notebook to extract, combine and transform data</p>

In [2]:
import pandas as pd
import numpy as np
import os 

<h2>Manipulate Happiness Data</h2>

In [30]:
html = pd.read_html("happiest_cities.html")

In [31]:
happiness_df = html[0]

In [34]:
happiness_df = happiness_df.rename(columns = {'City': 'Location'})

happiness_df['state'] = happiness_df['Location'].apply(lambda s: s.split(",")[-1])
happiness_df['city'] = happiness_df['Location'].apply(lambda s: s.split(",")[0])

happiness_df = happiness_df.drop(columns = ['Location'])

<h2>Manipulate Transportation Data</h2>

In [72]:
locations_df = pd.read_csv("EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [73]:
# still deciding what to do with names

# locations_df.rename(columns = {"GEOID10": "2010_fips_code", "GEOID20" : "2018_fips_code", 
#                     "STATEFP": 'state_fips_code', "COUNTYFP": "county_fips_code", 
#                     "Ac_Total": 'total_acres', 'Ac_Land': 'land_acres', 'Ac_Water': 'water_acres', 
#                     "Ac_Unpr": "unprotected_acres", "CountHU": "total_housing_units", "HH": "households", 
#                     "P_WrkAge": 'percent_working_age', 'AutoOwn0': 'owns_no_cars', 
#                     'Pct_AO0': 'percent_owns_no_cars', "AutoOwn1": 'owns_one_car', "Pct_AO1": "percent_owns_one_car",
#                     "AutoOwn2p": "owns_two_plus_cars", "Pct_AO2": 'percent_owns_2_plus_cars'})

<h3>Remove Rows Where Location Is Missing</h3>

In [74]:
locations_df = locations_df[pd.isnull(locations_df['CBSA_Name']) == False]

<h3>Get City and State Names</h3>

In [75]:
locations_df['CBSA_Name'] = locations_df['CBSA_Name'].astype(str)

In [76]:
locations_df['metro_area_cities'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[0])
locations_df['metro_area_states'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[-1])

locations_df['city'] = locations_df['metro_area_cities'].apply(lambda s: s.split("-"))
locations_df['state'] = locations_df['metro_area_states'].apply(lambda s: s.split("-"))

In [77]:
locations_df = locations_df.drop(columns = ['metro_area_cities', 'metro_area_states'])

<h3>Explode</h3>

Credit to <a href = "https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe">this</a> Stack Overflow post for helping me figure out how to do a multicolumn explode. 

In [78]:
cols = [c for c in list(locations_df.columns) if c not in ['city', 'state']]

In [ ]:
locations_df.set_index(cols).apply(pd.Series.explode).reset_index()

In [60]:
locations_df_long = locations_df.explode('city').explode('state')

In [61]:
locations_df_long.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,Shape_Area,City,State,metro,state,metro_cities,metro_states,metro_area_cities,metro_area_states,city
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,297836.0831,Dallas-Fort Worth-Arlington,TX,Dallas-Fort Worth-Arlington,TX,"[Dallas, Fort Worth, Arlington]",TX,Dallas-Fort Worth-Arlington,TX,Dallas
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,297836.0831,Dallas-Fort Worth-Arlington,TX,Dallas-Fort Worth-Arlington,TX,"[Dallas, Fort Worth, Arlington]",TX,Dallas-Fort Worth-Arlington,TX,Fort Worth
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,297836.0831,Dallas-Fort Worth-Arlington,TX,Dallas-Fort Worth-Arlington,TX,"[Dallas, Fort Worth, Arlington]",TX,Dallas-Fort Worth-Arlington,TX,Arlington
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,484945.1466,Dallas-Fort Worth-Arlington,TX,Dallas-Fort Worth-Arlington,TX,"[Dallas, Fort Worth, Arlington]",TX,Dallas-Fort Worth-Arlington,TX,Dallas
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,484945.1466,Dallas-Fort Worth-Arlington,TX,Dallas-Fort Worth-Arlington,TX,"[Dallas, Fort Worth, Arlington]",TX,Dallas-Fort Worth-Arlington,TX,Fort Worth
